In [16]:
%pip install tensorflow numpy nltk scikit-learn
%python -m nltk.downloader punkt
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


Actual code implementation using ResNet

In [8]:
import numpy as np
import tensorflow as tf
import os
import pickle
import string
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Add, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
def extract_features(image_directory, save_path, batch_size=32):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    encoder_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

    features = {}
    img_names = os.listdir(image_directory)
    for i in range(0, len(img_names), batch_size):
        batch_names = img_names[i:i+batch_size]
        batch_images = []
        for img_name in batch_names:
            img_path = os.path.join(image_directory, img_name)
            try:
                img = load_img(img_path, target_size=(224, 224))
                img = img_to_array(img)
                img = preprocess_input(img)
                batch_images.append(img)
            except Exception as e:
                print(f"Error processing {img_name}: {e}")
                continue
        if batch_images:
            batch_images = np.array(batch_images)
            batch_features = encoder_model.predict(batch_images)
            for j, img_name in enumerate(batch_names):
                features[os.path.splitext(img_name)[0]] = batch_features[j]

    with open(save_path, 'wb') as f:
        pickle.dump(features, f)
        

def clean_captions(caption_file):
    captions = {}
    with open(caption_file, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',', 1)
            if len(parts) < 2:
                continue
            img_id, caption = parts
            img_id = img_id.split('.')[0]
            caption = caption.lower()
            caption = caption.translate(str.maketrans('', '', string.punctuation))
            caption = "start " + caption + " end"
            if img_id not in captions:
                captions[img_id] = []
            captions[img_id].append(caption)
    return captions


def create_tokenizer(captions, max_vocab_size=5000):
    caption_list = [caption for caption_group in captions.values() for caption in caption_group]
    tokenizer = Tokenizer(num_words=max_vocab_size, oov_token='<unk>')
    tokenizer.fit_on_texts(caption_list)
    return tokenizer




In [ ]:
def extract_single_feature(image_path):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    encoder_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    feature = encoder_model.predict(img).flatten()
    return feature

In [ ]:
def build_image_captioning_model(vocab_size, max_length, feature_vector_size=2048, embedding_dim=256, lstm_units=256):
    image_input = Input(shape=(feature_vector_size,))
    image_dense = Dense(embedding_dim, activation='relu')(image_input)
    caption_input = Input(shape=(max_length,))
    caption_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(caption_input)
    caption_lstm = LSTM(lstm_units)(caption_embedding)
    decoder_input = Add()([image_dense, caption_lstm])
    decoder_output = Dense(vocab_size, activation='softmax')(decoder_input)
    model = Model(inputs=[image_input, caption_input], outputs=decoder_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def data_gen(captions, features, tokenizer, max_length, vocab_size, batch_size=32):
    feature_keys = set(features.keys())
    while True:
        X1, X2, y = [], [], []
        for img_id, caption_list in captions.items():
            if img_id not in feature_keys:
                continue
            for caption in caption_list:
                seq = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                    X1.append(features[img_id])
                    X2.append(in_seq)
                    y.append(out_seq)
                    if len(X1) == batch_size:
                        yield (np.array(X1), np.array(X2)), np.array(y)
                        X1, X2, y = [], [], []

In [19]:
def generate_caption_beam_search(model, tokenizer, image_feature, max_length, beam_width=3):
    start_token = 'start'
    end_token = 'end'

    beam = [(start_token, 0.0)] 

    for _ in range(max_length):
        new_beam = []
        
        for seq, score in beam:
            sequence = tokenizer.texts_to_sequences([seq])[0]
            sequence = pad_sequences([sequence], maxlen=max_length, padding='post')

            yhat = model.predict([np.array([image_feature]), sequence], verbose=0)
            yhat = yhat[0]

            # Get top `beam_width` candidates
            top_indices = np.argsort(yhat)[-beam_width:]
            for index in top_indices:
                word = tokenizer.index_word.get(index)
                if word is None:
                    continue

                new_seq = seq + ' ' + word
                new_score = score + np.log(yhat[index])
                new_beam.append((new_seq, new_score))
        
        new_beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_width]
        beam = new_beam

        if all(end_token in seq for seq, _ in beam):
            break

    final_sequence = beam[0][0]
    final_sequence = final_sequence.replace(start_token, '').replace(end_token, '').strip()
    return final_sequence


In [ ]:
def evaluate_bleu_score(model, tokenizer, features, captions, max_length):
    references, candidates = [], []
    for img_id, ground_truth_captions in captions.items():
        if img_id not in features:
            continue
        image_feature = features[img_id]
        generated_caption = generate_caption_beam_search(model, tokenizer, image_feature, max_length)
        references.append([caption.split() for caption in ground_truth_captions])
        candidates.append(generated_caption.split())
    bleu1 = corpus_bleu(references, candidates, weights=(1.0, 0, 0, 0))
    bleu2 = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0))
    print(f"BLEU-1 Score: {bleu1:.4f}")
    print(f"BLEU-2 Score: {bleu2:.4f}")

# Split data into train, validation, and test sets
def split_data(captions, test_size=0.2, val_size=0.1):
    image_ids = list(captions.keys())
    train_ids, temp_ids = train_test_split(image_ids, test_size=(test_size + val_size), random_state=42)
    val_ids, test_ids = train_test_split(temp_ids, test_size=test_size / (test_size + val_size), random_state=42)
    return (
        {img_id: captions[img_id] for img_id in train_ids},
        {img_id: captions[img_id] for img_id in val_ids},
        {img_id: captions[img_id] for img_id in test_ids},
    )

In [10]:
image_directory = './archive/images'
caption_file = './archive/captions.txt'
save_path = './features_resnet.pkl'

extract_features(image_directory, save_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [20]:
captions = clean_captions(caption_file)
train_captions, val_captions, test_captions = split_data(captions, test_size=0.1, val_size=0.1)

tokenizer = create_tokenizer(train_captions)
vocab_size = len(tokenizer.word_index) + 1
all_captions = [caption for caption_group in train_captions.values() for caption in caption_group]
max_length = max(len(tokenizer.texts_to_sequences([caption])[0]) for caption in all_captions)

with open(save_path, 'rb') as f:
    features = pickle.load(f)

batch_size = 32
steps_per_epoch = sum(len(caption_list) for caption_list in train_captions.values()) // batch_size

# Build and train the model
model = build_image_captioning_model(vocab_size, max_length)

# Early Stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model.fit(
    data_gen(train_captions, features, tokenizer, max_length, vocab_size, batch_size),
    steps_per_epoch=steps_per_epoch,
    validation_data=data_gen(val_captions, features, tokenizer, max_length, vocab_size, batch_size),
    validation_steps=len(val_captions) // batch_size,
    epochs=50,
    verbose=1,
    callbacks=[early_stopping]
)

model.save('image_captioning_model_resnet.keras')
with open('tokenizer_resnet.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

test_image_path = './test_images/mountain.jpg'
image_feature = extract_single_feature(test_image_path)
caption = generate_caption_beam_search(model, tokenizer, image_feature, max_length, beam_width=3)
print("Generated Caption:", caption)

evaluate_bleu_score(model, tokenizer, features, test_captions, max_length)

Epoch 1/50


/Users/david/Desktop/AI-ML-assignment/.venv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_736', 'keras_tensor_738']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1011/1011 ━━━━━━━━━━━━━━━━━━━━ 90s 88ms/step - accuracy: 0.1996 - loss: 5.4442 - val_accuracy: 0.2862 - val_loss: 4.3044
Epoch 2/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 87s 86ms/step - accuracy: 0.2937 - loss: 4.1119 - val_accuracy: 0.3125 - val_loss: 3.8207
Epoch 3/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 93s 92ms/step - accuracy: 0.3163 - loss: 3.7984 - val_accuracy: 0.3075 - val_loss: 3.7332
Epoch 4/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 89s 88ms/step - accuracy: 0.3165 - loss: 3.7607 - val_accuracy: 0.3537 - val_loss: 3.5429
Epoch 5/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 88s 87ms/step - accuracy: 0.3352 - loss: 3.6225 - val_accuracy: 0.3025 - val_loss: 3.7832
Epoch 6/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 91s 90ms/step - accuracy: 0.3389 - loss: 3.5579 - val_accuracy: 0.3237 - val_loss: 3.4849
Epoch 7/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 89s 88ms/step - accuracy: 0.3428 - loss: 3.5441 - val_accuracy: 0.3225 - val_loss: 3.5050
Epoch 8/50
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 97s 96ms/step - accuracy: 0.3497 - loss: 3.48